In [1]:
# Import necessary libraries
import pandas as pd
import os
import pycountry

# 1. Extracting Data
## Write code to extract the 31 country files from the emissions dataset that you are using for the project.

In [2]:
# Dataframe for sector
sector_data = {'sectorName': ['agriculture', 'buildings', 'fluorinated_gases', 'fossil_fuel_operations', 'manufacturing', 'mineral_extraction', 'power', 'waste']}
dimSector = pd.DataFrame(sector_data)

dimSector

,sectorName
0,agriculture
1,buildings
2,fluorinated_gases
3,fossil_fuel_operations
4,manufacturing
5,mineral_extraction
6,power
7,waste


In [3]:
# Dataframe for subsector
subsector_data = {
    'subsectorName': [
        # Agriculture
        'cropland-fires', 'enteric-fermentation', 'manure-management',
        'other-agricultural-soil-emissions', 'rice-cultivation', 'synthetic-fertilizer-application',
        
        # Buildings
        'other-onsite-fuel-usage', 'residential-and-commercial-onsite-fuel-usage',
        
        # Fluorinated Gases
        'fluorinated-gases',
        
        # Fossil Fuel Operations
        'coal-mining', 'oil-and-gas-production-and-transport', 'oil-and-gas-refining',
        'other-fossil-fuel-operations', 'solid-fuel-transformation',
        
        # Manufacturing
        'aluminum', 'cement', 'chemicals', 'other-manufacturing', 'pulp-and-paper', 'steel',
        
        # Mineral Extraction
        'bauxite-mining', 'copper-mining', 'iron-mining', 'rock-quarrying', 'sand-quarrying',
        
        # Power
        'electricity-generation', 'other-energy-use',
        
        # Waste
        'biological-treatment-of-solid-waste-&-biogenic', 'incineration-and-open-burning-of-waste',
        'solid-waste-disposal', 'wastewater-treatment-and-discharge'
    ],
    'sectorName': [
        # Agriculture
        'agriculture', 'agriculture', 'agriculture',
        'agriculture', 'agriculture', 'agriculture',
        
        # Buildings
        'buildings', 'buildings',
        
        # Fluorinated Gases
        'fluorinated_gases',
        
        # Fossil Fuel Operations
        'fossil_fuel_operations', 'fossil_fuel_operations', 'fossil_fuel_operations',
        'fossil_fuel_operations', 'fossil_fuel_operations',
        
        # Manufacturing
        'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing', 'manufacturing',
        
        # Mineral Extraction
        'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction', 'mineral_extraction',
        
        # Power
        'power', 'power',
        
        # Waste
        'waste', 'waste', 'waste', 'waste'
    ]
}

dimSubSector = pd.DataFrame(subsector_data)

# join sector df
dimSubSector = dimSubSector.merge(dimSector, on='sectorName')

# rename sector column
dimSubSector = dimSubSector.rename(columns={'sectorName':'sectorID'})

# # map sector name to sector ID
dict_sector = {name:i + 1 for i, name in enumerate(dimSector['sectorName'])}
dimSubSector['sectorID'] = dimSubSector['sectorID'].map(dict_sector)

dimSubSector = dimSubSector[['sectorID', 'subsectorName']]

dimSubSector.head(10)

,sectorID,subsectorName
0,1,cropland-fires
1,1,enteric-fermentation
2,1,manure-management
3,1,other-agricultural-soil-emissions
4,1,rice-cultivation
5,1,synthetic-fertilizer-application
6,2,other-onsite-fuel-usage
7,2,residential-and-commercial-onsite-fuel-usage
8,3,fluorinated-gases
9,4,coal-mining


In [4]:
# Combined csv will all the data
main_directory = 'original_data/'
dataframes = []

for root, dirs, files in os.walk(main_directory):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head()

,iso3_country,start_time,end_time,original_inventory_sector,gas,emissions_quantity,emissions_quantity_units,temporal_granularity,created_date,modified_date
0,MKD,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
1,MKD,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,ch4,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
2,MLI,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
3,MLI,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,ch4,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN
4,MLT,2020-01-01 00:00:00,2020-12-31 00:00:00,other-agricultural-soil-emissions,co2,0.0,tonnes,NaN,2022-09-06 12:39:52.76017,NaN


In [5]:
# Dataframe for country
dimCountry = combined_df['iso3_country'].unique()
dimCountry = pd.DataFrame(dimCountry, columns=['countryCode'])

def get_country_name(code):
    try:
        return pycountry.countries.get(alpha_3=code).name
    except AttributeError:
        return 'Unknown'

dimCountry['countryName'] = dimCountry['countryCode'].apply(get_country_name)

dimCountry

,countryCode,countryName
0,MKD,North Macedonia
1,MLI,Mali
2,MLT,Malta
3,MMR,Myanmar
4,ABW,Aruba
...,...,...
246,VAT,Holy See (Vatican City State)
247,CCK,Cocos (Keeling) Islands
248,CXR,Christmas Island
249,SGS,South Georgia and the South Sandwich Islands


In [12]:
# Dataframe for gas
dimGas = combined_df['gas'].unique()
dimGas = pd.DataFrame(dimGas, columns=['gasName'])

descriptions = [
    'Carbon dioxide',
    'Methane',
    'Nitrous oxide',
    'Equivalent CO2 impact over 20 years',
    'Equivalent CO2 impact over 100 years'
]

dimGas['gasDescription'] = descriptions

dimGas

,gasName,gasDescription
0,co2,Carbon dioxide
1,ch4,Methane
2,n2o,Nitrous oxide
3,co2e_20yr,Equivalent CO2 impact over 20 years
4,co2e_100yr,Equivalent CO2 impact over 100 years


In [13]:
# Dataframe for year 
unique_years = combined_df['start_time'].astype(str).str[:4].unique()
unique_years = sorted(unique_years)
dimYear = pd.DataFrame(unique_years, columns=['yearName'])

dimYear

,yearName
0,2015
1,2016
2,2017
3,2018
4,2019
5,2020
6,2021
7,2022


# 2.1. Cleaning the data

In [14]:
# fix countries that were not mapped correctly
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryCode,countryName
221,SCG,Unknown
245,XKX,Unknown


In [15]:
dimCountry.loc[dimCountry['countryCode'] == 'SCG', 'countryName'] = 'Serbia and Montenegro'
dimCountry.loc[dimCountry['countryCode'] == 'XKX', 'countryName'] = 'Republic of Kosovo'
dimCountry.loc[dimCountry['countryCode'] == 'TUR', 'countryName'] = 'Turkey'

# should now have no rows
dimCountry[dimCountry['countryName'] == 'Unknown']

,countryCode,countryName


In [16]:
# check if any rows have null values (for relevant columns only)
combined_df.iloc[:, :-4].isna().sum()

iso3_country                     0
start_time                       0
end_time                         0
original_inventory_sector        0
gas                              0
emissions_quantity           30002
dtype: int64

In [17]:
# replace null emission quantities with 0
combined_df['emissions_quantity'] = combined_df['emissions_quantity'].fillna(0)

# check to make sure it worked (should now be 0 for all)
combined_df.iloc[:, :-4].isna().sum()

iso3_country                 0
start_time                   0
end_time                     0
original_inventory_sector    0
gas                          0
emissions_quantity           0
dtype: int64

# 2.2. Transforming the data

In [18]:
# Dataframe for emission fact table

# organize data
fctEmission = combined_df.iloc[:, :-4]
fctEmission = fctEmission.drop('end_time', axis=1)
fctEmission['start_time'] = fctEmission['start_time'].astype(str).str[:4]
fctEmission

# rename columns
fctEmission = fctEmission.rename(columns={
    'iso3_country': 'countryID',
    'start_time':'yearID',
    'original_inventory_sector':'subsectorID',
    'gas':'gasID',
    'emissions_quantity':'emissionQuantity'
    })

# create dictionaries to map the dimension IDs to the fact table
dict_country = {country_code : i + 1 for i, country_code in enumerate(dimCountry['countryCode'])}
dict_year = {year : i + 1 for i, year in enumerate(dimYear['yearName'])}
dict_subsector = {subsector : i + 1 for i, subsector in enumerate(dimSubSector['subsectorName'])}
dict_gas = {gas : i + 1 for i, gas in enumerate(dimGas['gasName'])}

# map based on dimension tables
fctEmission['countryID'] = fctEmission['countryID'].map(dict_country)
fctEmission['yearID'] = fctEmission['yearID'].map(dict_year)
fctEmission['subsectorID'] = fctEmission['subsectorID'].map(dict_subsector)
fctEmission['gasID'] = fctEmission['gasID'].map(dict_gas)

# rearrange columns to match our dimensional model
fctEmission = fctEmission[['countryID', 'subsectorID', 'yearID', 'gasID' ,'emissionQuantity']]

fctEmission.head(20)

,countryID,subsectorID,yearID,gasID,emissionQuantity
0,1,4,6,1,0.0
1,1,4,6,2,0.0
2,2,4,6,1,0.0
3,2,4,6,2,0.0
4,3,4,6,1,0.0
5,3,4,6,2,0.0
6,4,4,6,1,0.0
7,5,4,6,1,0.0
8,5,4,6,3,0.0
9,5,4,6,2,0.0


# 3. Loading the data to the data warehouse

In [19]:
# export data as CSV files
combined_df.to_csv('transformed_data/combined_dataframe.csv', index=False)
dimSector.to_csv('transformed_data/dimSector.csv', index=False)
dimCountry.to_csv('transformed_data/dimCountry.csv', index=False)
dimGas.to_csv('transformed_data/dimGas.csv', index=False)
dimSubSector.to_csv('transformed_data/dimSubSector.csv', index=False)
dimYear.to_csv('transformed_data/dimYear.csv', index=False)
fctEmission.to_csv('transformed_data/fctEmission.csv', index=False)